In [0]:
import json
from glob import glob
from functions.sanity import validate_settings, initialize_schemas_and_volumes, initialize_empty_tables
from functions.utility import apply_job_type

layers = [
    ("layer_01_bronze", "bronze"),
    ("layer_01_bronze_history", "bronze_history"),
    ("layer_02_silver", "silver"),
    ("layer_02_silver_samples", "silver_samples"),
    ("layer_03_gold", "gold"),
]

job_settings = {color: [] for _, color in layers}
silver_parallel = []
silver_sequential = []

for layer_path, color in layers:
    paths = glob(f'./{layer_path}/*.json')
    for path in paths:
        table_name = path.split('/')[-1].split('.')[0]
        with open(path) as f:
            settings = json.load(f)
        settings = apply_job_type(settings)
        entry = {'table': table_name}
        if color == 'silver':
            requires = settings.get('requires', [])
            if requires:
                entry['requires'] = requires
                silver_sequential.append(entry)
            else:
                silver_parallel.append(entry)
        job_settings[color].append(entry)

for key, value in job_settings.items():
    dbutils.jobs.taskValues.set(key=key, value=value)

dbutils.jobs.taskValues.set(key='silver_parallel', value=silver_parallel)
dbutils.jobs.taskValues.set(key='silver_sequential', value=silver_sequential)

# Sanity check
validate_settings(dbutils)
initialize_schemas_and_volumes(spark)
initialize_empty_tables(spark)

